# Gesture Recognition
In this group project, you are going to build a 3D Conv model that will be able to predict the 5 gestures correctly. Please import the following libraries to get started.

In [1]:
import numpy as np
import os
from scipy.misc import imread, imresize
import datetime
import os

We set the random seed so that the results don't vary drastically.

In [2]:
np.random.seed(30)
import random as rn
rn.seed(30)
from keras import backend as K
import tensorflow as tf
tf.set_random_seed(30)

Using TensorFlow backend.


In this block, you read the folder names for training and validation. You also set the `batch_size` here. Note that you set the batch size in such a way that you are able to use the GPU in full capacity. You keep increasing the batch size until the machine throws an error.

In [3]:
train_doc = np.random.permutation(open('./Project_data/train.csv').readlines())
val_doc = np.random.permutation(open('./Project_data/val.csv').readlines())
batch_size = 10 #experiment with the batch size

In [4]:
print(len(train_doc))

663


In [5]:
print(len(val_doc))

100


### Preprocessing

In [6]:
# Helper Functions and our current preprogessing pipeline
# pre process ===> conbert_to_grayscale --> normalize_data

# Convert to grayscale
def convert_to_grayscale(data):
    return data.mean(axis=-1,keepdims=1) 

def normalize_data(data):
    return data/127.5-1

def preprocess_pipeline(data):
    return normalize_data(convert_to_grayscale(data))

## Generator
This is one of the most important part of the code. The overall structure of the generator has been given. In the generator, you are going to preprocess the images as you have images of 2 different dimensions as well as create a batch of video frames. You have to experiment with `img_idx`, `y`,`z` and normalization such that you get high accuracy.

In [7]:
def generator(source_path, folder_list, batch_size):
    
    x = 30 # number of frames
    y = 120 # image width
    z = 120 # image height
    
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    img_idx = [x for x in range(0,x,2)] #create a list of image numbers you want to use for a particular video
    while True:
        t = np.random.permutation(folder_list)
        num_batches = len(folder_list)//batch_size # calculate the number of batches
        for batch in range(num_batches): # we iterate over the number of batches
            batch_data = np.zeros((batch_size,x,y,z,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    
                    temp = imresize(image,(120,120))
                    temp = normalize_data(temp)
                    
                    batch_data[folder,idx,:,:,0] = (temp[:,:,0])
                    batch_data[folder,idx,:,:,1] = (temp[:,:,1])
                    batch_data[folder,idx,:,:,2] = (temp[:,:,2])
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do

        
        # write the code for the remaining data points which are left after full batches
        
        if (len(folder_list) != batch_size*num_batches):
            print("Batch: ",num_batches+1,"Index:", batch_size)
            batch_size = len(folder_list) - (batch_size*num_batches)
            batch_data = np.zeros((batch_size,x,y,z,c)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    temp = imresize(image,(120,120))
                    temp = normalize_data(temp)
                    
                    batch_data[folder,idx,:,:,0] = (temp[:,:,0])
                    batch_data[folder,idx,:,:,1] = (temp[:,:,1])
                    batch_data[folder,idx,:,:,2] = (temp[:,:,2])
                   
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels


Note here that a video is represented above in the generator as (number of images, height, width, number of channels). Take this into consideration while creating the model architecture.

In [8]:
curr_dt_time = datetime.datetime.now()
train_path = './Project_data/train'
val_path = './Project_data/val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs = 10# choose the number of epochs
print ('# epochs =', num_epochs)

# training sequences = 663
# validation sequences = 100
# epochs = 10


## Model
Here you make the model using different functionalities that Keras provides. Remember to use `Conv3D` and `MaxPooling3D` and not `Conv2D` and `Maxpooling2D` for a 3D convolution model. You would want to use `TimeDistributed` while building a Conv2D + RNN model. Also remember that the last layer is the softmax. Design the network in such a way that the model is able to give good accuracy on the least number of parameters so that it can fit in the memory of the webcam.

In [9]:
c = 3
nb_filters = [8,16,32,64]
nb_dense = [1000, 500, 5]
input_shape = (30, 120, 120, c)

In [10]:
from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation, Dropout
from keras.layers.convolutional import Conv3D, MaxPooling3D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers

#write your model here

model_a = Sequential()

model_a.add(Conv3D(nb_filters[0], 
                 kernel_size=(3,3,3), 
                 input_shape=input_shape,
                 padding='same'))
model_a.add(BatchNormalization())
model_a.add(Activation('relu'))

model_a.add(MaxPooling3D(pool_size=(2,2,2)))

model_a.add(Conv3D(nb_filters[1], 
                 kernel_size=(3,3,3), 
                 padding='same'))
model_a.add(BatchNormalization())
model_a.add(Activation('relu'))

model_a.add(MaxPooling3D(pool_size=(2,2,2)))

model_a.add(Conv3D(nb_filters[2], 
                 kernel_size=(1,3,3), 
                 padding='same'))
model_a.add(BatchNormalization())
model_a.add(Activation('relu'))

model_a.add(MaxPooling3D(pool_size=(2,2,2)))

model_a.add(Conv3D(nb_filters[3], 
                 kernel_size=(1,3,3), 
                 padding='same'))
model_a.add(BatchNormalization())
model_a.add(Activation('relu'))

model_a.add(MaxPooling3D(pool_size=(2,2,2)))

#Flatten Layers
model_a.add(Flatten())

model_a.add(Dense(nb_dense[0], activation='relu'))
model_a.add(Dropout(0.5))

model_a.add(Dense(nb_dense[1], activation='relu'))
model_a.add(Dropout(0.5))

#softmax layer
model_a.add(Dense(nb_dense[2], activation='softmax'))


Now that you have written the model, the next step is to `compile` the model. When you print the `summary` of the model, you'll see the total number of parameters you have to train.

In [11]:
rate = 0.001
optimiser = optimizers.Adam(lr=rate) #write your optimizer
model_a.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model_a.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_1 (Conv3D)            (None, 30, 120, 120, 8)   656       
_________________________________________________________________
batch_normalization_1 (Batch (None, 30, 120, 120, 8)   32        
_________________________________________________________________
activation_1 (Activation)    (None, 30, 120, 120, 8)   0         
_________________________________________________________________
max_pooling3d_1 (MaxPooling3 (None, 15, 60, 60, 8)     0         
_________________________________________________________________
conv3d_2 (Conv3D)            (None, 15, 60, 60, 16)    3472      
_________________________________________________________________
batch_normalization_2 (Batch (None, 15, 60, 60, 16)    64        
_________________________________________________________________
activation_2 (Activation)    (None, 15, 60, 60, 16)    0         
__________

Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.

In [12]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [13]:
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, cooldown=1, verbose=1)# write the Reducelronplateau code here
callbacks_list = [checkpoint, LR]

The `steps_per_epoch` and `validation_steps` are used by `fit_generator` to decide the number of next() calls it need to make.

In [14]:

if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

Let us now fit the model. This will start training the model and with the help of the checkpoints, you'll be able to save the model at the end of each epoch.

In [15]:
model_a.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  ./Project_data/val ; batch size = 10
Source path =  ./Project_data/train ; batch size = 10
Epoch 1/10


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:18: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:23: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


65/67 [============================>.] - ETA: 3s - loss: 5.7957 - categorical_accuracy: 0.2662Batch:  67 Index: 10


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:44: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:48: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


67/67 [==============================] - 127s 2s/step - loss: 5.7062 - categorical_accuracy: 0.2647 - val_loss: 1.4927 - val_categorical_accuracy: 0.4000

Epoch 00001: saving model to model_init_2020-12-2104_40_49.443651/model-00001-5.74708-0.26395-1.49271-0.40000.h5
Epoch 2/10
67/67 [==============================] - 14s 208ms/step - loss: 1.9267 - categorical_accuracy: 0.3284 - val_loss: 1.7683 - val_categorical_accuracy: 0.3700

Epoch 00002: saving model to model_init_2020-12-2104_40_49.443651/model-00002-1.92671-0.32836-1.76834-0.37000.h5
Epoch 3/10
67/67 [==============================] - 15s 217ms/step - loss: 1.5844 - categorical_accuracy: 0.3184 - val_loss: 1.2554 - val_categorical_accuracy: 0.4700

Epoch 00003: saving model to model_init_2020-12-2104_40_49.443651/model-00003-1.58445-0.31841-1.25539-0.47000.h5
Epoch 4/10
67/67 [==============================] - 14s 209ms/step - loss: 1.4098 - categorical_accuracy: 0.3881 - val_loss: 1.1900 - val_categorical_accuracy: 0.5000

Ep

##### The validation accuracy is 57%. Let us try some more models with different combination

In [16]:

nb_frames = 30 # number of frames
nb_rows = 120 # image width
nb_cols = 120 # image height 

nb_classes = 5
nb_channel = 3

### New Generator
- Let us try a new generator function with the preprocess pipeline function that we have defined above to convert the images to greyscale and then normalize it and then feed. Let us try to improve the performance

In [17]:
def new_generator(source_path, folder_list, batch_size):
      
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    img_idx = [x for x in range(0,nb_frames)] #create a list of image numbers you want to use for a particular video
    while True:
        t = np.random.permutation(folder_list)
        num_batches = len(folder_list)//batch_size # calculate the number of batches
        for batch in range(num_batches): # we iterate over the number of batches
            batch_data = np.zeros((batch_size,nb_frames,nb_rows,nb_cols,nb_channel)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    temp = imresize(image,(nb_rows,nb_cols))
                    #temp = normalize_data(temp)
                    temp = preprocess_pipeline(temp)
                    
                    batch_data[folder,idx] = temp

       
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do

        
        # write the code for the remaining data points which are left after full batches
        if (len(folder_list) != batch_size*num_batches):
            print("Batch: ",num_batches+1,"Index:", batch_size)
            batch_size = len(folder_list) - (batch_size*num_batches)
            batch_data = np.zeros((batch_size,nb_frames,nb_rows,nb_cols,nb_channel)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    temp = imresize(image,(nb_rows,nb_cols))
                    #temp = normalize_data(temp)
                    temp = preprocess_pipeline(temp)
                    
                    batch_data[folder,idx] = temp
                   
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels

##### Importing Adam optimizer

In [19]:
from keras.losses import categorical_crossentropy
from keras.optimizers import Adam

### Model 2:
- Using 2 Conv3D layers with filter as 32 then Max Pooling to extract features and dropout of 0.25 to avoid overfitting
- Using 2 Conv3D layers with filter as 64 then Max Pooling to extract features and dropout of 0.25 to avoid overfitting
- Kernel size is (3,3,3)
- Feeding it to MLP architecture. One Dense layer with 'Relu' and final with Softmax activation

In [53]:


# Define model
model = Sequential()
model.add(Conv3D(32, kernel_size=(3, 3, 3), input_shape=(nb_frames,nb_rows,nb_cols,nb_channel), padding='same'))
model.add(Activation('relu'))
model.add(Conv3D(32, kernel_size=(3, 3, 3), padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling3D(pool_size=(3, 3, 3), padding='same'))
model.add(Dropout(0.25))

model.add(Conv3D(64, kernel_size=(3, 3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv3D(64, kernel_size=(3, 3, 3), padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling3D(pool_size=(3, 3, 3), padding='same'))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(nb_classes, activation='softmax'))

model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_13 (Conv3D)           (None, 30, 120, 120, 32)  2624      
_________________________________________________________________
activation_13 (Activation)   (None, 30, 120, 120, 32)  0         
_________________________________________________________________
conv3d_14 (Conv3D)           (None, 30, 120, 120, 32)  27680     
_________________________________________________________________
activation_14 (Activation)   (None, 30, 120, 120, 32)  0         
_________________________________________________________________
max_pooling3d_13 (MaxPooling (None, 10, 40, 40, 32)    0         
_________________________________________________________________
dropout_8 (Dropout)          (None, 10, 40, 40, 32)    0         
_________________________________________________________________
conv3d_15 (Conv3D)           (None, 10, 40, 40, 64)    55360     
__________

- train_generator and val_generator to be used in .fit_generator

In [54]:
train_generator = new_generator(train_path, train_doc, batch_size)
val_generator = new_generator(val_path, val_doc, batch_size)

- Fitting the model

In [55]:
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  ./Project_data/val ; batch size = 10
Source path =  Epoch 1/10
./Project_data/train ; batch size = 10


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:14: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:18: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


64/67 [===========================>..] - ETA: 6s - loss: 1.7857 - categorical_accuracy: 0.2031Batch:  67 Index: 10


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:38: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:42: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


67/67 [==============================] - 162s 2s/step - loss: 1.7778 - categorical_accuracy: 0.1956 - val_loss: 1.6096 - val_categorical_accuracy: 0.2100

Epoch 00001: saving model to model_init_2020-12-2013_19_00.364355/model-00001-1.77960-0.19759-1.60958-0.21000.h5
Epoch 2/10
67/67 [==============================] - 46s 681ms/step - loss: 1.6093 - categorical_accuracy: 0.2189 - val_loss: 1.6089 - val_categorical_accuracy: 0.2100

Epoch 00002: saving model to model_init_2020-12-2013_19_00.364355/model-00002-1.60934-0.21891-1.60889-0.21000.h5
Epoch 3/10
67/67 [==============================] - 46s 681ms/step - loss: 1.6131 - categorical_accuracy: 0.1393 - val_loss: 1.6081 - val_categorical_accuracy: 0.2100

Epoch 00003: saving model to model_init_2020-12-2013_19_00.364355/model-00003-1.61311-0.13930-1.60806-0.21000.h5
Epoch 4/10
67/67 [==============================] - 46s 680ms/step - loss: 1.6101 - categorical_accuracy: 0.2040 - val_loss: 1.6085 - val_categorical_accuracy: 0.2100

Ep

##### The result is 21% on validation set, which very ver less. So, we have to try different model

### Model 3:
- Using 2 Conv3D layers with filter as 32 then Max Pooling to extract features and dropout of 0.25 to avoid overfitting
- Using 2 Conv3D layers with filter as 64 then Max Pooling to extract features and dropout of 0.25 to avoid overfitting
- Using 2 Conv3D layers with filter as 64 then Max Pooling to extract features and dropout of 0.25 to avoid overfitting
- Kernel Size is (3,3,3)
- Feeding it to MLP architecture. One Dense layer with 'Relu', adding batch normalization followed by dropout of 0.5 and final with Softmax activation

In [56]:
# Define model
model = Sequential()
model.add(Conv3D(32, kernel_size=(3, 3, 3), input_shape=(nb_frames,nb_rows,nb_cols,nb_channel), padding="same"))
model.add(Activation('relu'))
model.add(Conv3D(32, padding="same", kernel_size=(3, 3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling3D(pool_size=(3, 3, 3), padding="same"))
model.add(Dropout(0.25))

model.add(Conv3D(64, padding="same", kernel_size=(3, 3, 3)))
model.add(Activation('relu'))
model.add(Conv3D(64, padding="same", kernel_size=(3, 3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling3D(pool_size=(3, 3, 3), padding="same"))
model.add(Dropout(0.25))

model.add(Conv3D(64, padding="same", kernel_size=(3, 3, 3)))
model.add(Activation('relu'))
model.add(Conv3D(64, padding="same", kernel_size=(3, 3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling3D(pool_size=(3, 3, 3), padding="same"))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(nb_classes, activation='softmax'))
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_17 (Conv3D)           (None, 30, 120, 120, 32)  2624      
_________________________________________________________________
activation_17 (Activation)   (None, 30, 120, 120, 32)  0         
_________________________________________________________________
conv3d_18 (Conv3D)           (None, 30, 120, 120, 32)  27680     
_________________________________________________________________
activation_18 (Activation)   (None, 30, 120, 120, 32)  0         
_________________________________________________________________
max_pooling3d_15 (MaxPooling (None, 10, 40, 40, 32)    0         
_________________________________________________________________
dropout_11 (Dropout)         (None, 10, 40, 40, 32)    0         
_________________________________________________________________
conv3d_19 (Conv3D)           (None, 10, 40, 40, 64)    55360     
__________

- train_generator and val_generator to be used in .fit_generator

In [57]:
train_generator = new_generator(train_path, train_doc, batch_size)
val_generator = new_generator(val_path, val_doc, batch_size)

- Fitting the model

In [58]:
num_epochs = 20
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  ./Project_data/val ; batch size = 10
Source path =  ./Project_data/train ; batch size = 10
Epoch 1/20


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:14: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:18: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


56/67 [========================>.....] - ETA: 19s - loss: 1.9250 - categorical_accuracy: 0.3411Batch:  67 Index: 10


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:38: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:42: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


67/67 [==============================] - 121s 2s/step - loss: 1.9096 - categorical_accuracy: 0.3404 - val_loss: 2.2734 - val_categorical_accuracy: 0.2700

Epoch 00001: saving model to model_init_2020-12-2013_19_00.364355/model-00001-1.90882-0.34389-2.27336-0.27000.h5
Epoch 2/20
67/67 [==============================] - 46s 681ms/step - loss: 1.8820 - categorical_accuracy: 0.2836 - val_loss: 5.9526 - val_categorical_accuracy: 0.2400

Epoch 00002: saving model to model_init_2020-12-2013_19_00.364355/model-00002-1.88200-0.28358-5.95262-0.24000.h5
Epoch 3/20
67/67 [==============================] - 45s 678ms/step - loss: 1.7650 - categorical_accuracy: 0.3184 - val_loss: 2.4464 - val_categorical_accuracy: 0.2700

Epoch 00003: saving model to model_init_2020-12-2013_19_00.364355/model-00003-1.76496-0.31841-2.44636-0.27000.h5

Epoch 00003: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
Epoch 4/20
67/67 [==============================] - 45s 677ms/step - loss: 1.7410 - categ

##### The result is 41% on the valdation set, which ver less too. So, creating a new model

### Model 4:
- Creating a new model with 4 conv3D layers and adding batch activation before each 'relu' activation fucntion, folowed by Max pooling
- Adding 2 dense layers with 'relu' activation and 0.5 dropouts and final softmax layer. 

In [59]:
input_shape=(nb_frames,nb_rows,nb_cols,nb_channel)

# Define model
model = Sequential()

model.add(Conv3D(nb_filters[0], 
                 kernel_size=(3,3,3), 
                 input_shape=input_shape,
                 padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(MaxPooling3D(pool_size=(2,2,2)))

model.add(Conv3D(nb_filters[1], 
                 kernel_size=(3,3,3), 
                 padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(MaxPooling3D(pool_size=(2,2,2)))

model.add(Conv3D(nb_filters[2], 
                 kernel_size=(1,3,3), 
                 padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(MaxPooling3D(pool_size=(2,2,2)))

model.add(Conv3D(nb_filters[3], 
                 kernel_size=(1,3,3), 
                 padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(MaxPooling3D(pool_size=(2,2,2)))

#Flatten Layers
model.add(Flatten())

model.add(Dense(nb_dense[0], activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(nb_dense[1], activation='relu'))
model.add(Dropout(0.5))

#softmax layer
model.add(Dense(nb_dense[2], activation='softmax'))
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_23 (Conv3D)           (None, 30, 120, 120, 8)   656       
_________________________________________________________________
batch_normalization_13 (Batc (None, 30, 120, 120, 8)   32        
_________________________________________________________________
activation_23 (Activation)   (None, 30, 120, 120, 8)   0         
_________________________________________________________________
max_pooling3d_18 (MaxPooling (None, 15, 60, 60, 8)     0         
_________________________________________________________________
conv3d_24 (Conv3D)           (None, 15, 60, 60, 16)    3472      
_________________________________________________________________
batch_normalization_14 (Batc (None, 15, 60, 60, 16)    64        
_________________________________________________________________
activation_24 (Activation)   (None, 15, 60, 60, 16)    0         
__________

- fitting the model using train_generator and val_generator

In [60]:
train_generator = new_generator(train_path, train_doc, batch_size)
val_generator = new_generator(val_path, val_doc, batch_size)
num_epochs = 20
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  ./Project_data/val ; batch size = 10
Source path =  ./Project_data/trainEpoch 1/20
 ; batch size = 10


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:14: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:18: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


65/67 [============================>.] - ETA: 2s - loss: 5.3494 - categorical_accuracy: 0.3046Batch:  67 Index: 10


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:38: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:42: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


67/67 [==============================] - 86s 1s/step - loss: 5.2480 - categorical_accuracy: 0.3069 - val_loss: 5.9549 - val_categorical_accuracy: 0.2500

Epoch 00001: saving model to model_init_2020-12-2013_19_00.364355/model-00001-5.28812-0.30317-5.95491-0.25000.h5
Epoch 2/20
67/67 [==============================] - 24s 361ms/step - loss: 2.2719 - categorical_accuracy: 0.3134 - val_loss: 3.1975 - val_categorical_accuracy: 0.2800

Epoch 00002: saving model to model_init_2020-12-2013_19_00.364355/model-00002-2.27186-0.31343-3.19750-0.28000.h5
Epoch 3/20
67/67 [==============================] - 27s 407ms/step - loss: 1.7420 - categorical_accuracy: 0.3134 - val_loss: 1.4169 - val_categorical_accuracy: 0.3200

Epoch 00003: saving model to model_init_2020-12-2013_19_00.364355/model-00003-1.74199-0.31343-1.41688-0.32000.h5
Epoch 4/20
67/67 [==============================] - 29s 428ms/step - loss: 1.6395 - categorical_accuracy: 0.3035 - val_loss: 1.4706 - val_categorical_accuracy: 0.3600

Epo

##### The valiadtion accuracy is 67%, which is lot more than what we were getting till now. But let us try if it can be increased further

In [61]:
nb_frames = 30 # number of frames
nb_rows = 64 # image width
nb_cols = 64 # image height 

nb_classes = 5
nb_channel = 1

### Model 5:
Creating a new model by changing a few layers  

In [62]:
# Define model
model = Sequential()
model.add(Conv3D(32, kernel_size=(3, 3, 3), input_shape=(nb_frames,nb_rows,nb_cols,nb_channel), padding="same"))
model.add(Activation('relu'))
model.add(Conv3D(32, padding="same", kernel_size=(3, 3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling3D(pool_size=(3, 3, 3), padding="same"))
model.add(Dropout(0.25))

model.add(Conv3D(64, padding="same", kernel_size=(3, 3, 3)))
model.add(Activation('relu'))
model.add(Conv3D(64, padding="same", kernel_size=(3, 3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling3D(pool_size=(3, 3, 3), padding="same"))
model.add(Dropout(0.25))

model.add(Conv3D(64, padding="same", kernel_size=(3, 3, 3)))
model.add(Activation('relu'))
model.add(Conv3D(64, padding="same", kernel_size=(3, 3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling3D(pool_size=(3, 3, 3), padding="same"))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(nb_classes, activation='softmax'))
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_27 (Conv3D)           (None, 30, 64, 64, 32)    896       
_________________________________________________________________
activation_27 (Activation)   (None, 30, 64, 64, 32)    0         
_________________________________________________________________
conv3d_28 (Conv3D)           (None, 30, 64, 64, 32)    27680     
_________________________________________________________________
activation_28 (Activation)   (None, 30, 64, 64, 32)    0         
_________________________________________________________________
max_pooling3d_22 (MaxPooling (None, 10, 22, 22, 32)    0         
_________________________________________________________________
dropout_17 (Dropout)         (None, 10, 22, 22, 32)    0         
_________________________________________________________________
conv3d_29 (Conv3D)           (None, 10, 22, 22, 64)    55360     
__________

- Fitting the model with batch size 20 and number of epochs as 20

In [63]:
batch_size = 20
train_generator = new_generator(train_path, train_doc, batch_size)
val_generator = new_generator(val_path, val_doc, batch_size)
num_epochs = 20
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  Source path =  ./Project_data/train ; batch size = 20
Epoch 1/20
./Project_data/val ; batch size = 20


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:14: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:18: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


32/67 [=============>................] - ETA: 1:18 - loss: 1.6882 - categorical_accuracy: 0.3156Batch:  34 Index: 20


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:38: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:42: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


67/67 [==============================] - 85s 1s/step - loss: 1.5791 - categorical_accuracy: 0.3496 - val_loss: 3.0353 - val_categorical_accuracy: 0.1600

Epoch 00001: saving model to model_init_2020-12-2013_19_00.364355/model-00001-1.66145-0.32546-3.03525-0.16000.h5
Epoch 2/20
67/67 [==============================] - 26s 393ms/step - loss: 1.6166 - categorical_accuracy: 0.2537 - val_loss: 1.7599 - val_categorical_accuracy: 0.2300

Epoch 00002: saving model to model_init_2020-12-2013_19_00.364355/model-00002-1.61662-0.25373-1.75995-0.23000.h5
Epoch 3/20
67/67 [==============================] - 30s 445ms/step - loss: 1.5356 - categorical_accuracy: 0.3383 - val_loss: 3.5339 - val_categorical_accuracy: 0.1600

Epoch 00003: saving model to model_init_2020-12-2013_19_00.364355/model-00003-1.53556-0.33831-3.53389-0.16000.h5
Epoch 4/20
67/67 [==============================] - 30s 441ms/step - loss: 1.5494 - categorical_accuracy: 0.3085 - val_loss: 1.5636 - val_categorical_accuracy: 0.2300

Epo

##### The validation accuracy is 51% and it is still not saticfactory. 

### Model 6:
- Trying out the previous model with batch normalization after every conv3D layer but this time with batch size 10 and epochs as 30

In [20]:
nb_frames = 30 # number of frames
nb_rows = 120 # image width
nb_cols = 120 # image height 

nb_classes = 5
nb_channel = 1

input_shape=(nb_frames,nb_rows,nb_cols,nb_channel)

# Define model
model = Sequential()

model.add(Conv3D(nb_filters[0], 
                 kernel_size=(3,3,3), 
                 input_shape=input_shape,
                 padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(MaxPooling3D(pool_size=(2,2,2)))

model.add(Conv3D(nb_filters[1], 
                 kernel_size=(3,3,3), 
                 padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(MaxPooling3D(pool_size=(2,2,2)))

model.add(Conv3D(nb_filters[2], 
                 kernel_size=(1,3,3), 
                 padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(MaxPooling3D(pool_size=(2,2,2)))

model.add(Conv3D(nb_filters[3], 
                 kernel_size=(1,3,3), 
                 padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(MaxPooling3D(pool_size=(2,2,2)))

#Flatten Layers
model.add(Flatten())

model.add(Dense(nb_dense[0], activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(nb_dense[1], activation='relu'))
model.add(Dropout(0.5))

#softmax layer
model.add(Dense(nb_dense[2], activation='softmax'))
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_9 (Conv3D)            (None, 30, 120, 120, 8)   224       
_________________________________________________________________
batch_normalization_9 (Batch (None, 30, 120, 120, 8)   32        
_________________________________________________________________
activation_9 (Activation)    (None, 30, 120, 120, 8)   0         
_________________________________________________________________
max_pooling3d_9 (MaxPooling3 (None, 15, 60, 60, 8)     0         
_________________________________________________________________
conv3d_10 (Conv3D)           (None, 15, 60, 60, 16)    3472      
_________________________________________________________________
batch_normalization_10 (Batc (None, 15, 60, 60, 16)    64        
_________________________________________________________________
activation_10 (Activation)   (None, 15, 60, 60, 16)    0         
__________

- Fitting the model

In [21]:
batch_size = 10
train_generator = new_generator(train_path, train_doc, batch_size)
val_generator = new_generator(val_path, val_doc, batch_size)
num_epochs = 30
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  ./Project_data/val ; batch size = 10
Source path =  ./Project_data/train ; batch size = 10
Epoch 1/30


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:14: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:18: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


65/67 [============================>.] - ETA: 4s - loss: 5.6466 - categorical_accuracy: 0.2985Batch:  67 Index: 10


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:38: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:42: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


67/67 [==============================] - 143s 2s/step - loss: 5.6058 - categorical_accuracy: 0.2975 - val_loss: 5.6025 - val_categorical_accuracy: 0.2800

Epoch 00001: saving model to model_init_2020-12-2104_40_49.443651/model-00001-5.61891-0.29713-5.60247-0.28000.h5
Epoch 2/30
67/67 [==============================] - 24s 363ms/step - loss: 2.5152 - categorical_accuracy: 0.3433 - val_loss: 1.4799 - val_categorical_accuracy: 0.3900

Epoch 00002: saving model to model_init_2020-12-2104_40_49.443651/model-00002-2.51518-0.34328-1.47995-0.39000.h5
Epoch 3/30
67/67 [==============================] - 23s 349ms/step - loss: 1.7355 - categorical_accuracy: 0.3333 - val_loss: 1.2374 - val_categorical_accuracy: 0.4400

Epoch 00003: saving model to model_init_2020-12-2104_40_49.443651/model-00003-1.73550-0.33333-1.23735-0.44000.h5
Epoch 4/30
67/67 [==============================] - 26s 381ms/step - loss: 1.5360 - categorical_accuracy: 0.3383 - val_loss: 1.3706 - val_categorical_accuracy: 0.3500

Ep

##### The acuuracy does not change much with the changing of batch size and epochs. It is stillhaving 68% validation accuracy. 

##### Defining a random function

In [22]:
def get_random_affine():
    dx, dy = np.random.randint(-1.7, 1.8, 2)
    M = np.float32([[1, 0, dx], [0, 1, dy]])
    return M

In [23]:
import cv2

##### New Generator with data augmentation

In [28]:
def aug_generator(source_path, folder_list, batch_size):
      
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    img_idx = [x for x in range(0,nb_frames)] #create a list of image numbers you want to use for a particular video
    while True:
        t = np.random.permutation(folder_list)
        num_batches = len(folder_list)//batch_size # calculate the number of batches
        for batch in range(num_batches): # we iterate over the number of batches
            batch_data = np.zeros((batch_size,nb_frames,nb_rows,nb_cols,nb_channel)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            
            batch_data_aug = np.zeros((batch_size,nb_frames,nb_rows,nb_cols,nb_channel)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels_aug = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
  
            batch_data_aug2 = np.zeros((batch_size,nb_frames,nb_rows,nb_cols,nb_channel)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels_aug2 = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
 
            batch_data_aug3 = np.zeros((batch_size,nb_frames,nb_rows,nb_cols,nb_channel)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels_aug3 = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
      
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                M = get_random_affine()
                M2 = get_random_affine()
                M3 = get_random_affine()
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = cv2.imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item], cv2.IMREAD_COLOR)
                    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes   
                    resized = cv2.resize(image, (nb_rows,nb_cols), interpolation = cv2.INTER_AREA)
                    batch_data[folder,idx] = resized
                    batch_data_aug[folder,idx] = cv2.warpAffine(resized, M, (resized.shape[0], resized.shape[1]))
                    batch_data_aug2[folder,idx] = cv2.warpAffine(resized, M2, (resized.shape[0], resized.shape[1]))
                    batch_data_aug3[folder,idx] = cv2.warpAffine(resized, M3, (resized.shape[0], resized.shape[1]))
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
                batch_labels_aug[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
                batch_labels_aug2[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
                batch_labels_aug3[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            
            batch_data = np.append(batch_data, batch_data_aug, axis = 0) 
            batch_data = np.append(batch_data, batch_data_aug2, axis = 0) 
            batch_data = np.append(batch_data, batch_data_aug3, axis = 0)
            batch_labels = np.append(batch_labels, batch_labels_aug, axis = 0) 
            batch_labels = np.append(batch_labels, batch_labels_aug2, axis = 0) 
            batch_labels = np.append(batch_labels, batch_labels_aug3, axis = 0)
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do

        
        # write the code for the remaining data points which are left after full batches
        if (len(folder_list) != batch_size*num_batches):
            print("Batch: ",num_batches+1,"Index:", batch_size)
            batch_size = len(folder_list) - (batch_size*num_batches)
            
            batch_data = np.zeros((batch_size,nb_frames,nb_rows,nb_cols,nb_channel)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            
            batch_data_aug = np.zeros((batch_size,nb_frames,nb_rows,nb_cols,nb_channel)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels_aug = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
  
            batch_data_aug2 = np.zeros((batch_size,nb_frames,nb_rows,nb_cols,nb_channel)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels_aug2 = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
   
            batch_data_aug3 = np.zeros((batch_size,nb_frames,nb_rows,nb_cols,nb_channel)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels_aug3 = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
      
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                M = get_random_affine()
                M2 = get_random_affine()
                M3 = get_random_affine()
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = cv2.imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item], cv2.IMREAD_COLOR)
                    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    resized = cv2.resize(image, (nb_rows,nb_cols), interpolation = cv2.INTER_AREA)
                    batch_data[folder,idx] = resized
                    batch_data_aug[folder,idx] = cv2.warpAffine(resized, M, (resized.shape[0], resized.shape[1]))
                    batch_data_aug2[folder,idx] = cv2.warpAffine(resized, M2, (resized.shape[0], resized.shape[1]))
                    batch_data_aug3[folder,idx] = cv2.warpAffine(resized, M3, (resized.shape[0], resized.shape[1]))
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
                batch_labels_aug[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
                batch_labels_aug2[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
                batch_labels_aug3[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            
            batch_data = np.append(batch_data, batch_data_aug, axis = 0) 
            batch_data = np.append(batch_data, batch_data_aug2, axis = 0) 
            batch_data = np.append(batch_data, batch_data_aug3, axis = 0)
            batch_labels = np.append(batch_labels, batch_labels_aug, axis = 0) 
            batch_labels = np.append(batch_labels, batch_labels_aug2, axis = 0) 
            batch_labels = np.append(batch_labels, batch_labels_aug3, axis = 0) 
            yield batch_data, batch_labels

### Model 7: 

In [25]:
nb_frames = 30 # number of frames
nb_rows = 120 # image width
nb_cols = 120 # image height 

nb_classes = 5
nb_channel = 3

input_shape=(nb_frames,nb_rows,nb_cols,nb_channel)

# Define model
model = Sequential()

model.add(Conv3D(nb_filters[0], 
                 kernel_size=(3,3,3), 
                 input_shape=input_shape,
                 padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(MaxPooling3D(pool_size=(2,2,2)))

model.add(Conv3D(nb_filters[1], 
                 kernel_size=(3,3,3), 
                 padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(MaxPooling3D(pool_size=(2,2,2)))

model.add(Conv3D(nb_filters[2], 
                 kernel_size=(1,3,3), 
                 padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(MaxPooling3D(pool_size=(2,2,2)))

model.add(Conv3D(nb_filters[3], 
                 kernel_size=(1,3,3), 
                 padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(MaxPooling3D(pool_size=(2,2,2)))

#Flatten Layers
model.add(Flatten())

model.add(Dense(nb_dense[0], activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(nb_dense[1], activation='relu'))
model.add(Dropout(0.5))

#softmax layer
model.add(Dense(nb_dense[2], activation='softmax'))
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_13 (Conv3D)           (None, 30, 120, 120, 8)   656       
_________________________________________________________________
batch_normalization_13 (Batc (None, 30, 120, 120, 8)   32        
_________________________________________________________________
activation_13 (Activation)   (None, 30, 120, 120, 8)   0         
_________________________________________________________________
max_pooling3d_13 (MaxPooling (None, 15, 60, 60, 8)     0         
_________________________________________________________________
conv3d_14 (Conv3D)           (None, 15, 60, 60, 16)    3472      
_________________________________________________________________
batch_normalization_14 (Batc (None, 15, 60, 60, 16)    64        
_________________________________________________________________
activation_14 (Activation)   (None, 15, 60, 60, 16)    0         
__________

In [26]:
batch_size = 5
train_generator = aug_generator(train_path, train_doc, batch_size)
val_generator = aug_generator(val_path, val_doc, batch_size)
num_epochs = 20
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  ./Project_data/val ; batch size = 5
Source path =  ./Project_data/train ; batch size = 5
Epoch 1/20
67/67 [==============================] - 64s 963ms/step - loss: 4.4215 - categorical_accuracy: 0.2769 - val_loss: 1.8285 - val_categorical_accuracy: 0.3400

Epoch 00001: saving model to model_init_2020-12-2104_40_49.443651/model-00001-4.42153-0.27687-1.82849-0.34000.h5
Epoch 2/20
67/67 [==============================] - 59s 886ms/step - loss: 1.3948 - categorical_accuracy: 0.4228 - val_loss: 1.5687 - val_categorical_accuracy: 0.3150

Epoch 00002: saving model to model_init_2020-12-2104_40_49.443651/model-00002-1.39836-0.41994-1.56870-0.31500.h5
Epoch 3/20
67/67 [==============================] - 34s 505ms/step - loss: 1.3275 - categorical_accuracy: 0.4440 - val_loss: 1.0777 - val_categorical_accuracy: 0.5600

Epoch 00003: saving model to model_init_2020-12-2104_40_49.443651/model-00003-1.32751-0.44403-1.07774-0.56000.h5
Epoch 4/20
67/67 [==============================] - 3

##### It has a accuray of 84%. 

### Model 8: 

In [29]:
nb_frames = 30 # number of frames
nb_rows = 120 # image width
nb_cols = 120 # image height 

nb_classes = 5
nb_channel = 3

input_shape=(nb_frames,nb_rows,nb_cols,nb_channel)

# Define model
model1a = Sequential()

model1a.add(Conv3D(nb_filters[0], 
                 kernel_size=(3,3,3), 
                 input_shape=input_shape,
                 padding='same'))
model1a.add(BatchNormalization())
model1a.add(Activation('relu'))

model1a.add(MaxPooling3D(pool_size=(2,2,2)))

model1a.add(Conv3D(nb_filters[1], 
                 kernel_size=(3,3,3), 
                 padding='same'))
model1a.add(BatchNormalization())
model1a.add(Activation('relu'))

model1a.add(MaxPooling3D(pool_size=(2,2,2)))

model1a.add(Conv3D(nb_filters[2], 
                 kernel_size=(1,3,3), 
                 padding='same'))
model1a.add(BatchNormalization())
model1a.add(Activation('relu'))

model1a.add(MaxPooling3D(pool_size=(2,2,2)))

model1a.add(Conv3D(nb_filters[3], 
                 kernel_size=(1,3,3), 
                 padding='same'))
model1a.add(BatchNormalization())
model1a.add(Activation('relu'))

model1a.add(MaxPooling3D(pool_size=(2,2,2)))

#Flatten Layers
model1a.add(Flatten())

model1a.add(Dense(nb_dense[0], activation='relu'))
model1a.add(Dropout(0.5))

model1a.add(Dense(nb_dense[1], activation='relu'))
model1a.add(Dropout(0.5))

#softmax layer
model1a.add(Dense(nb_dense[2], activation='softmax'))
model1a.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model1a.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_17 (Conv3D)           (None, 30, 120, 120, 8)   656       
_________________________________________________________________
batch_normalization_17 (Batc (None, 30, 120, 120, 8)   32        
_________________________________________________________________
activation_17 (Activation)   (None, 30, 120, 120, 8)   0         
_________________________________________________________________
max_pooling3d_17 (MaxPooling (None, 15, 60, 60, 8)     0         
_________________________________________________________________
conv3d_18 (Conv3D)           (None, 15, 60, 60, 16)    3472      
_________________________________________________________________
batch_normalization_18 (Batc (None, 15, 60, 60, 16)    64        
_________________________________________________________________
activation_18 (Activation)   (None, 15, 60, 60, 16)    0         
__________

In [30]:
batch_size = 10
train_generator = aug_generator(train_path, train_doc, batch_size)
val_generator = aug_generator(val_path, val_doc, batch_size)
num_epochs = 20
model1a.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  ./Project_data/val ; batch size = 10
Source path =  ./Project_data/train ; batch size = 10
Epoch 1/20
67/67 [==============================] - 124s 2s/step - loss: 2.6555 - categorical_accuracy: 0.3173 - val_loss: 1.4686 - val_categorical_accuracy: 0.3225

Epoch 00001: saving model to model_init_2020-12-2104_40_49.443651/model-00001-2.66845-0.31712-1.46856-0.32250.h5
Epoch 2/20
67/67 [==============================] - 38s 568ms/step - loss: 1.6111 - categorical_accuracy: 0.3632 - val_loss: 1.2953 - val_categorical_accuracy: 0.4725

Epoch 00002: saving model to model_init_2020-12-2104_40_49.443651/model-00002-1.61110-0.36318-1.29532-0.47250.h5
Epoch 3/20
67/67 [==============================] - 37s 556ms/step - loss: 1.5472 - categorical_accuracy: 0.3383 - val_loss: 1.4417 - val_categorical_accuracy: 0.3000

Epoch 00003: saving model to model_init_2020-12-2104_40_49.443651/model-00003-1.54718-0.33831-1.44166-0.30000.h5
Epoch 4/20
67/67 [==============================] - 3

##### It has a validation accuracy of 74.5 %

##### Changing the Batch Size to 5 and Epochs to 30

In [31]:
batch_size = 5
train_generator = aug_generator(train_path, train_doc, batch_size)
val_generator = aug_generator(val_path, val_doc, batch_size)
num_epochs = 30
model1a.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  ./Project_data/val ; batch size = 5
Source path =  ./Project_data/train ; batch size = 5
Epoch 1/30
67/67 [==============================] - 63s 942ms/step - loss: 0.5654 - categorical_accuracy: 0.7896 - val_loss: 0.6765 - val_categorical_accuracy: 0.7750

Epoch 00001: saving model to model_init_2020-12-2104_40_49.443651/model-00001-0.56535-0.78955-0.67651-0.77500.h5
Epoch 2/30
67/67 [==============================] - 59s 874ms/step - loss: 0.6647 - categorical_accuracy: 0.7652 - val_loss: 0.5568 - val_categorical_accuracy: 0.7900

Epoch 00002: saving model to model_init_2020-12-2104_40_49.443651/model-00002-0.66812-0.76435-0.55681-0.79000.h5
Epoch 3/30
67/67 [==============================] - 35s 526ms/step - loss: 0.6184 - categorical_accuracy: 0.7562 - val_loss: 0.8051 - val_categorical_accuracy: 0.6050

Epoch 00003: saving model to model_init_2020-12-2104_40_49.443651/model-00003-0.61839-0.75622-0.80507-0.60500.h5
Epoch 4/30
67/67 [==============================] - 3


Epoch 00026: saving model to model_init_2020-12-2104_40_49.443651/model-00026-0.35451-0.87935-0.48768-0.83500.h5
Epoch 27/30
67/67 [==============================] - 34s 514ms/step - loss: 0.4422 - categorical_accuracy: 0.8358 - val_loss: 0.4902 - val_categorical_accuracy: 0.8000

Epoch 00027: saving model to model_init_2020-12-2104_40_49.443651/model-00027-0.44221-0.83582-0.49023-0.80000.h5

Epoch 00027: ReduceLROnPlateau reducing learning rate to 1.2207031829802872e-07.
Epoch 28/30
67/67 [==============================] - 35s 527ms/step - loss: 0.4069 - categorical_accuracy: 0.8570 - val_loss: 0.5715 - val_categorical_accuracy: 0.8350

Epoch 00028: saving model to model_init_2020-12-2104_40_49.443651/model-00028-0.40691-0.85697-0.57151-0.83500.h5
Epoch 29/30
67/67 [==============================] - 34s 511ms/step - loss: 0.4360 - categorical_accuracy: 0.8246 - val_loss: 0.6829 - val_categorical_accuracy: 0.7750

Epoch 00029: saving model to model_init_2020-12-2104_40_49.443651/model

##### It has validation accuracy of 87%. 

##### New generator with change in data augmentation 

In [32]:
def aug_generator(source_path, folder_list, batch_size):
      
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    img_idx = [x for x in range(0,nb_frames)] #create a list of image numbers you want to use for a particular video
    while True:
        t = np.random.permutation(folder_list)
        num_batches = len(folder_list)//batch_size # calculate the number of batches
        for batch in range(num_batches): # we iterate over the number of batches
            batch_data = np.zeros((batch_size,nb_frames,nb_rows,nb_cols,nb_channel)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            
            batch_data_aug = np.zeros((batch_size,nb_frames,nb_rows,nb_cols,nb_channel)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels_aug = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
  
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                M = get_random_affine()
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = cv2.imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item], cv2.IMREAD_COLOR)
                    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes   
                    resized = cv2.resize(image, (nb_rows,nb_cols), interpolation = cv2.INTER_AREA)
                    batch_data[folder,idx] = resized
                    batch_data_aug[folder,idx] = cv2.warpAffine(resized, M, (resized.shape[0], resized.shape[1]))
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
                batch_labels_aug[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            
            batch_data = np.append(batch_data, batch_data_aug, axis = 0) 
            batch_labels = np.append(batch_labels, batch_labels_aug, axis = 0) 
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do

        
        # write the code for the remaining data points which are left after full batches
        if (len(folder_list) != batch_size*num_batches):
            print("Batch: ",num_batches+1,"Index:", batch_size)
            batch_size = len(folder_list) - (batch_size*num_batches)
            
            batch_data = np.zeros((batch_size,nb_frames,nb_rows,nb_cols,nb_channel)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            
            batch_data_aug = np.zeros((batch_size,nb_frames,nb_rows,nb_cols,nb_channel)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels_aug = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output

            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                M = get_random_affine()
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = cv2.imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item], cv2.IMREAD_COLOR)
                    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    resized = cv2.resize(image, (nb_rows,nb_cols), interpolation = cv2.INTER_AREA)
                    batch_data[folder,idx] = resized
                    batch_data_aug[folder,idx] = cv2.warpAffine(resized, M, (resized.shape[0], resized.shape[1]))
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
                batch_labels_aug[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            
            batch_data = np.append(batch_data, batch_data_aug, axis = 0) 
            batch_labels = np.append(batch_labels, batch_labels_aug, axis = 0) 

            yield batch_data, batch_labels

#### Model 9: 

In [33]:
nb_frames = 30 # number of frames
nb_rows = 120 # image width
nb_cols = 120 # image height 

nb_classes = 5
nb_channel = 3

input_shape=(nb_frames,nb_rows,nb_cols,nb_channel)

# Define model
model = Sequential()

model.add(Conv3D(nb_filters[0], 
                 kernel_size=(3,3,3), 
                 input_shape=input_shape,
                 padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(MaxPooling3D(pool_size=(2,2,2)))

model.add(Conv3D(nb_filters[1], 
                 kernel_size=(3,3,3), 
                 padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(MaxPooling3D(pool_size=(2,2,2)))

model.add(Conv3D(nb_filters[2], 
                 kernel_size=(1,3,3), 
                 padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(MaxPooling3D(pool_size=(2,2,2)))

model.add(Conv3D(nb_filters[3], 
                 kernel_size=(1,3,3), 
                 padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(MaxPooling3D(pool_size=(2,2,2)))

#Flatten Layers
model.add(Flatten())

model.add(Dense(nb_dense[0], activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(nb_dense[1], activation='relu'))
model.add(Dropout(0.5))

#softmax layer
model.add(Dense(nb_dense[2], activation='softmax'))
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_21 (Conv3D)           (None, 30, 120, 120, 8)   656       
_________________________________________________________________
batch_normalization_21 (Batc (None, 30, 120, 120, 8)   32        
_________________________________________________________________
activation_21 (Activation)   (None, 30, 120, 120, 8)   0         
_________________________________________________________________
max_pooling3d_21 (MaxPooling (None, 15, 60, 60, 8)     0         
_________________________________________________________________
conv3d_22 (Conv3D)           (None, 15, 60, 60, 16)    3472      
_________________________________________________________________
batch_normalization_22 (Batc (None, 15, 60, 60, 16)    64        
_________________________________________________________________
activation_22 (Activation)   (None, 15, 60, 60, 16)    0         
__________

In [34]:
batch_size = 10
train_generator = aug_generator(train_path, train_doc, batch_size)
val_generator = aug_generator(val_path, val_doc, batch_size)
num_epochs = 20
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path = Source path =  ./Project_data/train ; batch size = 10
 Epoch 1/20
./Project_data/val ; batch size = 10
67/67 [==============================] - 70s 1s/step - loss: 3.5457 - categorical_accuracy: 0.3336 - val_loss: 3.3236 - val_categorical_accuracy: 0.3100

Epoch 00001: saving model to model_init_2020-12-2104_40_49.443651/model-00001-3.56637-0.33183-3.32358-0.31000.h5
Epoch 2/20
67/67 [==============================] - 20s 299ms/step - loss: 1.9838 - categorical_accuracy: 0.3159 - val_loss: 1.4836 - val_categorical_accuracy: 0.4200

Epoch 00002: saving model to model_init_2020-12-2104_40_49.443651/model-00002-1.98381-0.31592-1.48357-0.42000.h5
Epoch 3/20
67/67 [==============================] - 22s 326ms/step - loss: 1.5808 - categorical_accuracy: 0.4104 - val_loss: 1.1495 - val_categorical_accuracy: 0.5150

Epoch 00003: saving model to model_init_2020-12-2104_40_49.443651/model-00003-1.58078-0.41045-1.14946-0.51500.h5
Epoch 4/20
67/67 [==============================] - 21

##### The accuracy is 82.5%

##### Changing the Batch Size to 5 and Epoch to 20

In [35]:
batch_size = 5
train_generator = aug_generator(train_path, train_doc, batch_size)
val_generator = aug_generator(val_path, val_doc, batch_size)
num_epochs = 20
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  ./Project_data/val ; batch size = 5
Source path =  ./Project_data/train ; batch size = 5
Epoch 1/20
67/67 [==============================] - 35s 521ms/step - loss: 0.5239 - categorical_accuracy: 0.8119 - val_loss: 0.5179 - val_categorical_accuracy: 0.7900

Epoch 00001: saving model to model_init_2020-12-2104_40_49.443651/model-00001-0.52389-0.81194-0.51787-0.79000.h5
Epoch 2/20
67/67 [==============================] - 33s 490ms/step - loss: 0.5445 - categorical_accuracy: 0.8090 - val_loss: 0.5725 - val_categorical_accuracy: 0.7400

Epoch 00002: saving model to model_init_2020-12-2104_40_49.443651/model-00002-0.54368-0.80967-0.57249-0.74000.h5
Epoch 3/20
67/67 [==============================] - 20s 296ms/step - loss: 0.6180 - categorical_accuracy: 0.7587 - val_loss: 0.4990 - val_categorical_accuracy: 0.8900

Epoch 00003: saving model to model_init_2020-12-2104_40_49.443651/model-00003-0.61803-0.75871-0.49895-0.89000.h5
Epoch 4/20
67/67 [==============================] - 1

##### It has a vaidation accuracy of 89%. 

##### Changing the batch size and epochs

In [36]:
batch_size = 20
train_generator = aug_generator(train_path, train_doc, batch_size)
val_generator = aug_generator(val_path, val_doc, batch_size)
num_epochs = 20
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  ./Project_data/val ; batch size = 20
Source path =  ./Project_data/train ; batch size = 20
Epoch 1/20
67/67 [==============================] - 78s 1s/step - loss: 0.3725 - categorical_accuracy: 0.8719 - val_loss: 0.5239 - val_categorical_accuracy: 0.7775

Epoch 00001: saving model to model_init_2020-12-2104_40_49.443651/model-00001-0.36329-0.87861-0.52386-0.77750.h5
Epoch 2/20
67/67 [==============================] - 23s 350ms/step - loss: 0.4654 - categorical_accuracy: 0.8657 - val_loss: 0.5315 - val_categorical_accuracy: 0.7750

Epoch 00002: saving model to model_init_2020-12-2104_40_49.443651/model-00002-0.46543-0.86567-0.53146-0.77500.h5
Epoch 3/20
67/67 [==============================] - 25s 370ms/step - loss: 0.4629 - categorical_accuracy: 0.8234 - val_loss: 0.5250 - val_categorical_accuracy: 0.7825

Epoch 00003: saving model to model_init_2020-12-2104_40_49.443651/model-00003-0.46286-0.82338-0.52495-0.78250.h5

Epoch 00003: ReduceLROnPlateau reducing learning rate

##### It has a ccuracy of 78.35%. 

### Model 10: 

In [37]:
nb_frames = 30 # number of frames
nb_rows = 120 # image width
nb_cols = 120 # image height 

nb_classes = 5
nb_channel = 3

input_shape=(nb_frames,nb_rows,nb_cols,nb_channel)

# Define model
model = Sequential()

model.add(Conv3D(nb_filters[0], 
                 kernel_size=(3,3,3), 
                 input_shape=input_shape,
                 padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(MaxPooling3D(pool_size=(2,2,2)))

model.add(Conv3D(nb_filters[1], 
                 kernel_size=(3,3,3), 
                 padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(MaxPooling3D(pool_size=(2,2,2)))

model.add(Conv3D(nb_filters[2], 
                 kernel_size=(1,3,3), 
                 padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(MaxPooling3D(pool_size=(2,2,2)))

model.add(Conv3D(nb_filters[3], 
                 kernel_size=(1,3,3), 
                 padding='same'))
model.add(Activation('relu'))
model.add(Dropout(0.25))

model.add(MaxPooling3D(pool_size=(2,2,2)))

#Flatten Layers
model.add(Flatten())

model.add(Dense(nb_dense[0], activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(nb_dense[1], activation='relu'))
model.add(Dropout(0.5))

#softmax layer
model.add(Dense(nb_dense[2], activation='softmax'))
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_25 (Conv3D)           (None, 30, 120, 120, 8)   656       
_________________________________________________________________
batch_normalization_25 (Batc (None, 30, 120, 120, 8)   32        
_________________________________________________________________
activation_25 (Activation)   (None, 30, 120, 120, 8)   0         
_________________________________________________________________
max_pooling3d_25 (MaxPooling (None, 15, 60, 60, 8)     0         
_________________________________________________________________
conv3d_26 (Conv3D)           (None, 15, 60, 60, 16)    3472      
_________________________________________________________________
batch_normalization_26 (Batc (None, 15, 60, 60, 16)    64        
_________________________________________________________________
activation_26 (Activation)   (None, 15, 60, 60, 16)    0         
__________

In [38]:
batch_size = 10
train_generator = aug_generator(train_path, train_doc, batch_size)
val_generator = aug_generator(val_path, val_doc, batch_size)
num_epochs = 20
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path = Source path =  ./Project_data/val  ./Project_data/train ; batch size = 10
; batch size = 10
Epoch 1/20
67/67 [==============================] - 68s 1s/step - loss: 1.6741 - categorical_accuracy: 0.3657 - val_loss: 1.3837 - val_categorical_accuracy: 0.4150

Epoch 00001: saving model to model_init_2020-12-2104_40_49.443651/model-00001-1.68101-0.36425-1.38368-0.41500.h5
Epoch 2/20
67/67 [==============================] - 21s 316ms/step - loss: 1.4462 - categorical_accuracy: 0.3682 - val_loss: 1.7043 - val_categorical_accuracy: 0.3000

Epoch 00002: saving model to model_init_2020-12-2104_40_49.443651/model-00002-1.44618-0.36816-1.70431-0.30000.h5
Epoch 3/20
67/67 [==============================] - 20s 299ms/step - loss: 1.3578 - categorical_accuracy: 0.4552 - val_loss: 1.2815 - val_categorical_accuracy: 0.4500

Epoch 00003: saving model to model_init_2020-12-2104_40_49.443651/model-00003-1.35779-0.45522-1.28153-0.45000.h5
Epoch 4/20
67/67 [==============================] - 21

##### It has a accuracy of 74%. 

### So, the Model 9 with batch size 5 and number of epochs 20 is the best model. 